In [ ]:
pip install torch librosa transformers datasets torchaudio evaluate jiwer

In [18]:
from IPython import get_ipython
from IPython.display import display
from datasets import Dataset
from evaluate import load
from transformers import Wav2Vec2Processor, TrainingArguments, Trainer, Wav2Vec2ForCTC
from pydub import AudioSegment
from pydub.silence import split_on_silence

import json
import torch
import librosa
import torchaudio
import numpy as np
import IPython.display as ipd  # Jupyter Notebook에서 오디오 재생

# 데이터 생성을 위해 작업  
- 오디오 파일 분할 무음 구간을 기준으로 분리작업

In [ ]:
# 오디오 파일 로드
audio = AudioSegment.from_wav("../target_file/0002.wav")

# 무음 구간 기준으로 오디오 분할
chunks = split_on_silence(
    audio,
    min_silence_len=1000,  # 최소 무음 길이 (밀리초, 예: 1초)
    silence_thresh=-40,    # 무음으로 간주할 음량(dBFS, 예: -40dB)
    keep_silence=500       # 분할 후 무음의 일부를 유지 (밀리초)
)

# 분할된 오디오 파일 저장
for i, chunk in enumerate(chunks):
    chunk.export(f"./train/chunk_{i}.wav", format="wav")
    print(f"chunk_{i}.wav saved.")

- train.json 을 통하여 학습 데이터를 생성한다. 

In [ ]:
import os
import librosa

# 오디오 파일이 저장된 폴더 경로
AUDIO_DIR = "../train_file/"  # 여기에 오디오 파일이 들어 있음

# 샘플링 레이트 설정 (Wav2Vec2는 일반적으로 16kHz 사용)
TARGET_SR = 16000  

# 최대 길이 찾기
max_length_samples = 0  # 최대 샘플 수
max_length_seconds = 0  # 최대 길이 (초)
longest_audio = ""  # 가장 긴 파일명 저장

# 폴더 내 모든 .wav 파일 확인
for filename in os.listdir(AUDIO_DIR):
    if filename.endswith(".wav"):
        file_path = os.path.join(AUDIO_DIR, filename)
        
        # 오디오 파일 로드
        audio, sr = librosa.load(file_path, sr=TARGET_SR)
        
        # 현재 오디오의 길이 (샘플 수)
        num_samples = len(audio)
        
        # 현재 오디오의 길이 (초 단위)
        duration_seconds = num_samples / sr
        
        # 최대 길이 갱신
        if num_samples > max_length_samples:
            max_length_samples = num_samples
            max_length_seconds = duration_seconds
            longest_audio = filename

# 결과 출력
print(f" 가장 긴 오디오 파일: {longest_audio}")
print(f" 최대 샘플 수: {max_length_samples} samples")
print(f" 최대 길이: {max_length_seconds:.2f} 초")

 가장 긴 오디오 파일: chunk_61.wav
 최대 샘플 수: 188849 samples
 최대 길이: 11.80 초


# 데이터 생성

- 가장 긴 샘플수 에 맞춰서 192000 맞춰서 무음 패딩진행

- 모델 : kresnik/wav2vec2-large-xlsr-korean

In [13]:
# JSON 데이터 로드
with open("../train_file/train.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Dataset 형태로 변환
dataset = Dataset.from_list(data)

# Wav2Vec2 Processor 로드
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")

# max_length 설정 (12초 기준)
MAX_LENGTH = 192000  # 12초 (16kHz 샘플링 기준)

# 오디오 전처리 함수 (패딩 및 attention_mask 추가)
def preprocess_data(batch):
    audio_path = batch["audio_path"]

    # 오디오 파일 로드
    waveform, sample_rate = librosa.load(audio_path, sr=16000)
    waveform = torch.tensor(waveform)


    # 오디오 길이 확인
    num_samples = len(waveform)

    # 패딩 적용 (짧은 오디오 → 12초 길이로 맞춤)
    if num_samples < MAX_LENGTH:
        pad_length = MAX_LENGTH - num_samples
        waveform = torch.nn.functional.pad(waveform, (0, pad_length), mode="constant")
        attention_mask = [1] * num_samples + [0] * pad_length  # 실제 음성 부분: 1, 패딩 부분: 0
    else:
        waveform = waveform[:MAX_LENGTH]  # 12초보다 길면 잘라냄
        attention_mask = [1] * MAX_LENGTH  # 모든 부분이 실제 음성

    # Wav2Vec2 입력값 변환 (고정된 길이 적용)
    input_values = processor(
        waveform.numpy(), sampling_rate=16000, return_tensors="pt"
    ).input_values.squeeze(0)

    # 텍스트를 토큰 ID로 변환 (패딩 적용)
    labels = processor.tokenizer(
        batch["text"], padding="max_length", max_length=256, truncation=True
    ).input_ids

    labels = [-100 if token == processor.tokenizer.pad_token_id else token for token in labels]

    return {"input_values": input_values, "labels": labels, "attention_mask": attention_mask}


In [14]:
dataset = dataset.map(preprocess_data, remove_columns=["audio_path", "text"])
dataset_split = dataset.train_test_split(test_size=0.2)
train_dataset = dataset_split["train"]
val_dataset = dataset_split["test"]

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

Map: 100%|██████████| 74/74 [00:01<00:00, 72.43 examples/s] 


In [15]:
print(train_dataset)
print(val_dataset)

Dataset({
    features: ['input_values', 'labels', 'attention_mask'],
    num_rows: 59
})
Dataset({
    features: ['input_values', 'labels', 'attention_mask'],
    num_rows: 15
})


# 데이터 확인 

In [ ]:
# 첫 번째 샘플 가져오기
sample = dataset[1]

# Tensor에서 NumPy 배열로 변환
audio_np = np.array(sample["input_values"])  # 모델 입력값

# 샘플링 레이트 설정 (16kHz)
sampling_rate = 16000

# 오디오 재생
ipd.Audio(audio_np, rate=sampling_rate)

In [26]:
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer
import torch
device = "cuda"
# 1️⃣ 모델 및 프로세서 로드
model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to(device)
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
#for name, param in model.named_parameters():
#    print(f"{name}: requires_grad={param.requires_grad}")

# 학습 메트릭을 확인하기 위해서 wer 매트릭을 설정한다.

In [27]:
wer_metric = load("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = torch.argmax(torch.tensor(pred_logits), dim=-1)
    pred_texts = processor.batch_decode(pred_ids)
    label_texts = processor.batch_decode(pred.label_ids)

    wer = wer_metric.compute(predictions=pred_texts, references=label_texts)
    return {"wer": wer}

In [19]:
def data_collator(data):
    batch = {
        "input_values": torch.stack([torch.tensor(f["input_values"]) for f in data]),  # input_values 사용
        "labels": torch.tensor([f["labels"] for f in data]),  # Tensor 변환
        "attention_mask": torch.tensor([f["attention_mask"] for f in data]),  # 패딩 정보
    }
    return batch

# 모델 학습 시작
- ../target_file/0002.wav 파일을 무음 단위로 분리 하여 학습을 진행했습니다.
- train_loss 와 valid_loss가 현저히 줄어드는 것을 확인했습니다.

중간에 더 이상 wer이 줄어들지 않는 것을 확인하여 강제 중단 하였습니다.

In [35]:
from transformers import TrainingArguments, Trainer

# ✅ 학습 설정 (수정된 버전)
training_args = TrainingArguments(
    output_dir="./wav2vec2_finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,  # gradient accumulation 추가
    learning_rate=5e-4,  # 학습률 증가 (기본값: 5e-5 ~ 1e-4)
    warmup_steps=500,  # 적절한 warmup 추가
    save_steps=500,
    eval_steps=500,
    num_train_epochs=100,
    weight_decay=0.005,
    save_strategy="epoch",
    logging_steps=10,
    push_to_hub=False,
    report_to="none",
    dataloader_pin_memory=True,
    dataloader_num_workers=0,  # 데이터 로딩 속도 개선
    group_by_length=True,  # 길이 기반 batching
    eval_strategy="epoch",  # `evaluation_strategy` → `eval_strategy` 변경
)

# Trainer 설정 (수정된 버전)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
)
# 학습 시작
trainer.train()

# 모델 저장
model.save_pretrained("./wav2vec2_finetuned")
processor.save_pretrained("./wav2vec2_finetuned")

<ipython-input-35-e86cfd571791>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Wer
1,No log,4.761405,0.625000
2,3.243900,4.354398,0.625000
3,3.276400,3.861022,0.625000
4,2.742000,3.457585,0.625000
5,2.450700,2.957560,0.612500
6,2.450700,2.477921,0.575000
7,2.010400,2.119188,0.550000
8,1.708300,1.813997,0.550000
9,1.407300,1.511225,0.525000
10,1.030000,1.127399,0.450000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

KeyboardInterrupt: 

# 허깅페이스로 모델을 업로드 

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

model = Wav2Vec2ForCTC.from_pretrained("./wav2vec2_finetuned/checkpoint-176")
processor = Wav2Vec2Processor.from_pretrained("./finetuned_processor")

from huggingface_hub import HfApi

model_name = "naseunghoo/wav2vec2-korean-finetuned"  # 원하는 모델명 설정
api = HfApi()

# Hugging Face Hub에 업로드
model.push_to_hub(model_name)
processor.push_to_hub(model_name)

# 모델 학습전과 후 모델 비교 
첫번째 모델은 허깅페이스 한글로 만들어진 모델 사용 
두번째 모델은 직접 파인튜닝해서 모델 허깅페이스에 업로드 해서 확인 

- 첫번째 모델 :kresnik/wav2vec2-large-xlsr-korean
- 두번째 모델 :naseunghoo/wav2vec2-large-xlsr-korean 


첫번째 모델은 단어를 유추하지 못하거나 다른 언어를 추론하지 못했습니다. 
두번쨰 모델은 해당 음성 파일을 학습시켜서 해당 단어를 유추했지만 [UNK] 토큰에 대한 처리가 부족했습니다. 

In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

device = "mps"

# Wav2Vec2 모델 및 프로세서 로드
processor = Wav2Vec2Processor.from_pretrained("kresnik/wav2vec2-large-xlsr-korean")
model = Wav2Vec2ForCTC.from_pretrained("kresnik/wav2vec2-large-xlsr-korean").to(device)

# 오디오 파일 로드 및 Resampling
def load_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)

    # Resampling (16kHz로 변환)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)
    return waveform, 16000

# Wav2Vec2를 사용한 음성 → 텍스트 변환
def speech_to_text(audio_path):
    waveform, sample_rate = load_audio(audio_path)

    # Wav2Vec2 입력 형식으로 변환 (GPU에서 실행)
    input_values = processor(
        waveform.squeeze(), return_tensors="pt", sampling_rate=sample_rate
    ).input_values.to(device)  # GPU로 이동

    # 너무 긴 입력 방지
    if input_values.shape[1] > 300000:
        print("입력 크기 초과! 잘라서 처리 확인")
        input_values = input_values[:, :300000]

    # 모델을 통해 예측 수행 (GPU에서 실행)
    with torch.no_grad():
        logits = model(input_values).logits

    # 예측값을 텍스트로 변환
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription

# 내 음성 파일 학습 결과 출력
audio_file = "../target_file/0002.wav"  # 학습할 음성 파일
result = speech_to_text(audio_file)
print(f"음성 인식 결과: {result}")

입력 크기 초과! 잘라서 처리 확인
음성 인식 결과: 누슨 건강한가요 내 건강해어디를 닥쳤어요 무릎게 가졌어언제 닥쳤어요 한 을길 전이었니요


In [24]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

device = "mps"

# Wav2Vec2 모델 및 프로세서 로드
processor = Wav2Vec2Processor.from_pretrained("naseunghoo/wav2vec2-korean-finetuned")
model = Wav2Vec2ForCTC.from_pretrained("naseunghoo/wav2vec2-korean-finetuned").to(device)

# 오디오 파일 로드 및 Resampling
def load_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)

    # Resampling (16kHz로 변환)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)
    return waveform, 16000

# Wav2Vec2를 사용한 음성 → 텍스트 변환
def speech_to_text(audio_path):
    waveform, sample_rate = load_audio(audio_path)

    # Wav2Vec2 입력 형식으로 변환 (GPU에서 실행)
    input_values = processor(
        waveform.squeeze(), return_tensors="pt", sampling_rate=sample_rate
    ).input_values.to(device)  # GPU로 이동

    # 너무 긴 입력 방지
    if input_values.shape[1] > 300000:
        print("입력 크기 초과! 잘라서 처리 확인")
        input_values = input_values[:, :300000]

    # 모델을 통해 예측 수행 (GPU에서 실행)
    with torch.no_grad():
        logits = model(input_values).logits

    # 예측값을 텍스트로 변환
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription

# 내 음성 파일 학습 결과 출력
audio_file = "../target_file/0002.wav"  # 학습할 음성 파일
result = speech_to_text(audio_file)
print(f"음성 인식 결과: {result}")

입력 크기 초과! 잘라서 처리 확인
음성 인식 결과: 요즘 건강한가요네[unk] 건강해요[unk]  어디를 다쳤어요무릎이 끄졌어요[unk] 언제 다쳤어요한 일주일 전이요[unk] [unk]
